In [ ]:
from cyclomics import *
warnings.simplefilter("ignore")

In [ ]:
##USER INPUTS

#specify data folder
#data from https://zenodo.org/record/3925250/files/Cyclomics_manuscript.zip
data_folder = f'/Volumes/1TB/Cyclomics_manuscript/RCA'

#sample to process
sample_name = 'CY_LOT1_QC_0001_000'


show_plots   = True #show plots in this notebook (it can consume lots of RAM)

#Save plots to drive
save_fig     = True
save_in      = 'plots'


#Basic filtering
read_all = True #if True, read all the file (mind not running out of RAM)
lines_to_read = 50_000 #if not read_all, choose how many lines to read (1 read per line)
len_filter = 5_000 #bases

In [ ]:
##Run
sample_folder = f'{data_folder}/{sample_name}/'

if save_fig:
    os.mkdir(f'{sample_folder}/{save_in}')
    #!mkdir $sample_folder/$save_in

files = list_of_files(sample_folder, 'txt.gz', recursive=1)
structure_file = [f for f in files if 'structure.txt.gz' in f][0]



data = []
read_len_dist     = Counter() #all reads here
composition_dict  = {}        #all reads passing len_filter here

with gzip.open(structure_file, 'r') as f:
    line_number = 0
    for line_number, line in enumerate(f):
        if line_number == 0:
            continue
        #if not line.startswith('/'):
        #    continue
            
        d = line.decode()strip().split('\t')
        try:
            if d[1] == 'nan':
                read_len_dist.update({0})
                continue
        except Exception as e:
            print(e)
            print(line)
            print(line_number)
            break
            
        if len(d) > 1: #and line.startswith('/'):
            
            data.append(d[:2])
            read_len = sum([int(i.split(':')[0]) for i in d[1].split(',')])
            read_len_dist.update({read_len})
            
            if read_len >= len_filter:
                segments        = d[1].split(',')
                composition     = Counter([i.split(':')[1] for i in segments])
                
                for k in ['I','BB','U']:
                    if k not in composition:
                        composition[k] = 0
                composition_dict[d[0]] = composition
    
        else:
            read_len_dist.update({0})
        
        if not read_all:
            lines_to_read -= 1
            if not lines_to_read:
                break

In [ ]:
##Colormap normalization based on TP53 coordinates 17:7565097-7590856 (GRCh37)

#https://matplotlib.org/users/colormaps.html
cmap_0 = cm.gist_rainbow
cmap_1 = cm.prism #sharper difference between close numbers but not unique
cmap_2 = cm.tab20
cmap_chr = cm.hsv #for chromosomes

##From Cyclomics_TP53_assay.bed
#17	7572800	7573100
#17	7573800	7574100
#17	7576400	7577700
#17	7578100	7578600
#17	7579200	7580000

vmin=7572800 #TP53 mapping start
vmax=7580000 #TP53 mapping end

#normalize coordinates
norm = Normalize(vmin, vmax)
norm_chr = Normalize(1, 24) #for chromosomes


##Chr color map
plt.style.use('ggplot')
plt.figure(figsize=(20,4), dpi=200)
y = 0
r = list(range(1, 25))
for x in r:
    plt.scatter(x,y  , s=300, color=cmap_chr(norm_chr(x)), lw=1, edgecolors='black')
plt.xticks(r)
plt.title('Chromosomes color map')
plt.show()


##TP53 color cmap
#for style in plt.style.available:
plt.style.use('ggplot')
plt.figure(figsize=(20,4), dpi=200)
y = 0
r = range(vmin, vmax, 100)
for x in r:
    plt.scatter(x,y+1, s=100, color=cmap_0(norm(x)), lw=1, edgecolors='black')
    plt.scatter(x,y  , s=100, color=cmap_1(norm(x)), lw=1, edgecolors='black')
    plt.scatter(x,y-1, s=100, color=cmap_2(norm(x)), lw=1, edgecolors='black')
plt.title('TP53 color map')
plt.show()

In [ ]:
## Example
#structure = '63:U,188:BB:R:BB200_2:1:194:0,20:U,21:U,...,195:BB:R:BB200_2:2:194:0,182:U'


insert = 'I'
#backbones color chart
bb_colors = {
    'BB22':'RED',
    'BB24':'BLUE',
    'BB25':'GRAY',
    'BBCR':'ORANGE',
    'PJET':'GREEN'
}

with_multiple_BB = []
with_multiple_I = []
with_template_shift = {"None":[],
                       "Unexpected":[],
                       "Regular":[],
                       "??":[]}


#TP53 on GRCh37
#expected_chr = '17'
#vmin=7572800 #TP53 mapping start
#vmax=7580000 #TP53 mapping end



#TP53 sequence only
expected_chr = 'TP53'
vmin=0 #TP53 mapping start
vmax=25772 #TP53 mapping end


#BRAF on GRCh37
#expected_chr = '7'
#vmin = 140452000
#vmax = 140458000

add_label = False
label = None #defaul label

try:
    if not len(processed): #skip if already processed, added to resume the labeling
        label_dict = {0:'discard',
                      1:'keep_all',
                      2:'keep_longest_block',
                      3:'keep_most_common_insert',
                      4:'complex_fix_needed',
                      5:'remap',
                      6:'??'}
        label_string = '|'.join([f'{k}:{v}'for k,v in label_dict.items()])
        grouped_by_labels = {}
except NameError:
    processed = set()

actions = {} #for filtering debug    
    
#Filter reads
print_report = True
filter1_on    = True #Optimal reads
filter2_on    = False
filter3_on    = False
filter4_on    = False #BB only

min_repeats_filter = 5 #number of BB-I blocks
chr_filter_in = 'TP53'
chr_filter_out = 'BRAF'
max_unmapped = 500
filter_pass = [] 


insertID_count = Counter()


for n, (bam, structure) in enumerate(data):#[:1000]):
    
    bam = bam.split('/')[-1]
    if bam in processed:
        continue

    ##Desecting
    segments        = structure.split(',')
    composition     = Counter([i.split(':')[1] for i in segments])
    seg_len = segments_len(structure)
 
    mapped      = Counter(i.split(':')[3] for i in segments if i[-2:] !=':U')
    unmapped    = Counter('U' for i in segments if i[-2:] ==':U')
    forward_BB  = structure.count('BB:F')
    reverse_BB  = structure.count('BB:R')
    longer_unmapped_region = max(int(i.split(':')[0]) for i in segments if i[-2:] ==':U')    
    c_str           = collapse(structure, remove_dust=25, trim_edges=1) #collapsed structure
    c_blocks        = list(get_reps(c_str[0]))  #consecutive blocks

    dna_shift       = check_dna_shift(c_str, c_blocks)
    with_template_shift[dna_shift].append(bam)

    Iid             = Counter()
    BBid            = Counter()
    read_len        = 0
    for s in segments:
        i = s.split(':')
        if i[1] == 'I':
            Iid.update({i[-1]})
            
        elif i[1] == 'BB':
            BBid.update({i[-1]})
        read_len += int(i[0])

    if len(Iid) > 1:
        sorted_v = sorted(Iid.values()) 
        if sorted_v[0] > 5 and sorted_v[0]/sorted_v[0] < 3:
            insertID_count.update({len(Iid)})
        else:
            insertID_count.update({1})
    else:
        insertID_count.update({1})


    if len(BBid) > 1:
        with_multiple_BB.append(bam)
    if len(Iid) > 1:
        with_multiple_I.append(bam)


    ##Scoring
    len_s = len(segments)


    ##Filtering#########
    
    ##Filter 1##
    if filter1_on:
        if longer_unmapped_region > max_unmapped:
            continue
        if not mapped:
            continue
        elif chr_filter_in not in mapped or chr_filter_out in mapped:
            continue
        if 'I' not in composition or 'BB' not in composition:
            continue
        elif composition['I'] < min_repeats_filter or composition['BB'] < min_repeats_filter:
            continue
        elif not 0.75 <= (composition['I']/composition['BB']) <= 1.25:
            continue
        else:
            filter_pass.append(bam)
    ####################
    
    
    ##Filter 2##
    if filter2_on:
        if longer_unmapped_region > max_unmapped:
            continue
        elif composition['BB'] < min_repeats_filter:
            continue
        #only forward or only reverse BB in one read
        if forward_BB and reverse_BB:
            continue
        else:
            filter_pass.append(bam)
    ####################
    
    
    ##Filter 3##
    if filter3_on:
        if 'I' not in composition or 'BB' not in composition:
            continue
        if seg_len['I'] < 100: #Inserts of at least 100bp
            continue
        if seg_len['BB'] < 200: #BB of at least 200bp
            continue
        else:
            filter_pass.append(bam)
    ####################
    
    
    ##Filter 4##
    if filter4_on:
        if 'BB' not in composition or 'I' in composition:
            continue
        if seg_len['BB'] < 200: #BB of at least 200bp
            continue
        else:
            filter_pass.append(bam)
    ####################
    


    if show_plots:
        plot_structure(structure,
                       bb_colors,
                       insert,
                       expected_chr,
                       vmin,
                       vmax,
                       title=bam,
                       show=show_plots,
                       savefig=f'{sample_folder}{save_in}/{bam}.png' if save_fig else False
                      )

    if add_label:
        while True:
            label = int(input(f'Input one of the following labels:\n{label_string}\n'))
            if label in label_dict:
                break
            else:
                print('Wrong label value!')
        try:
            grouped_by_labels[label].append(bam)
        except KeyError:
            grouped_by_labels[label] = [bam]



    ##Filtering demo
    action = 'discard'
    if 'I' in composition and composition['I'] >= 5:
        action = 'remap'
        if 'BB' in composition:
            if 0.8 <= (composition['I']/composition['BB']) <= 1.4:
                action = 'keep'

    try:
        actions[action].append(bam)
    except KeyError:
        actions[action] = [bam]

    ##Report
    if print_report:
        print(f'seqence N    : {n}')
        print(f'label        : {label}')
        print(f'from         : {bam}')
        print(f'read length  : {read_len}')
        ####
        #Add alignment of consensus vs target mutation
        #Flag mutants expected/unexpected contaminant

        ####
        print(f'structure    : {structure}')
        print(f'segments_len : {seg_len}')
        print(f'c_structure  : {c_str}')
        print(f'c_blocks     : {c_blocks}')
        print(f'dna_shift    : {dna_shift}')
        print(f'segments     : {len(segments)}')
        print(f'composition  : {composition}')
        print(f'Insert   IDs : {Iid}')
        print(f'Backbone IDs : {BBid}')
        print(f'mapped       : {mapped}')
        print(f'unmapped     : {unmapped}')
        print(f'action       : {action}')
        print(f'\n{"-"*20}\n')
